In [44]:
!pip install pingouin


In [45]:
# these packages are often used in data analyses, so import them first. 
%matplotlib inline

# pandas for data process, seaborn for plotting, in addition, matplotlib
import numpy as np
import pandas as pd  # manage data
import seaborn as sns # statistical data visualization
import matplotlib.pyplot as plt
import pingouin as pg

## Repeated measures ANOVA of the mean correct rates with Experiment (1-4) 

In [46]:
#A repeated measures ANOVA of the mean correct rates with Experiment (1-4) 
dat = pd.read_csv('modelrlt/models/log_rstan/rlt/WMCrr.csv')
dat.head()

,Unnamed: 0,Exp,WMSize,NSub,m_WMCrr,n,se_WMCrr
0,1,Exp1,low,1,0.916667,120,0.025442
1,2,Exp1,low,2,0.925000,120,0.024246
2,3,Exp1,low,3,0.883333,120,0.029551
3,4,Exp1,low,4,0.866667,120,0.031292
4,5,Exp1,low,5,0.958333,120,0.018395


In [47]:
Crr_exp = dat.groupby('Exp').m_WMCrr.mean().round(2)
Crr_exp

Exp
Exp1    0.81
Exp2    0.76
Exp3    0.73
Exp4    0.70
Name: m_WMCrr, dtype: float64

In [48]:
pg.pairwise_ttests(data = dat, 
            dv = 'm_WMCrr',  within = 'Exp', subject = 'NSub', padjust = 'holm')

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Exp,Exp1,Exp2,True,True,2.132026,15.0,two-sided,0.049946,0.149837,holm,1.504,0.814075
1,Exp,Exp1,Exp3,True,True,5.867934,15.0,two-sided,0.000031,0.000186,holm,807.098,1.585307
2,Exp,Exp1,Exp4,True,True,5.638092,15.0,two-sided,0.000047,0.000236,holm,554.231,1.882971
3,Exp,Exp2,Exp3,True,True,2.121656,15.0,two-sided,0.050934,0.149837,holm,1.482,0.605428
4,Exp,Exp2,Exp4,True,True,4.882416,15.0,two-sided,0.000199,0.000796,holm,155.652,0.986116
5,Exp,Exp3,Exp4,True,True,1.703510,15.0,two-sided,0.109095,0.149837,holm,0.835,0.474697


LSD comparison showed significantly higher correct rates in Experiment 1 (mean of .81) than that in Experiments 2-4 (mean of .76, .73 and .70 respectively, all _ps < .025_)

In [49]:
dat.groupby('WMSize').m_WMCrr.mean()

WMSize
high      0.648307
low       0.874349
medium    0.725911
Name: m_WMCrr, dtype: float64

In [50]:
dat_new = dat.groupby(['WMSize','NSub']).agg({'m_WMCrr':'mean'})
dat_new.head()

m_WMCrr
WMSize NSub          
high   1     0.677083
       2     0.568750
       3     0.606250
       4     0.604167
       5     0.695833

In [51]:
dat['NewNSub'] = dat.Exp.str.slice(3, 4).astype(np.uint8)*16 + dat.NSub 

In [59]:
aov = pg.mixed_anova(dv='m_WMCrr', between='Exp',
                  within='WMSize', subject='NewNSub', data=dat)
aov.round(3)

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,Exp,0.331,3,60,0.110,11.152,0.000,0.358,NaN
1,WMSize,1.689,2,120,0.844,454.218,0.000,0.883,0.921
2,Interaction,0.023,6,120,0.004,2.040,0.066,0.093,NaN


In [70]:
# creating a dict file 
WMSize = {'low': 1,'medium': 2, 'high': 3}
# traversing through dataframe
# Gender column and writing
# values where key matches
dat['WMSize_num'] = [WMSize[item] for item in dat.WMSize]
dat.head()

,Unnamed: 0,Exp,WMSize,NSub,m_WMCrr,n,se_WMCrr,NewNSub,WMSize_num
0,1,Exp1,low,1,0.916667,120,0.025442,17,1
1,2,Exp1,low,2,0.925000,120,0.024246,18,1
2,3,Exp1,low,3,0.883333,120,0.029551,19,1
3,4,Exp1,low,4,0.866667,120,0.031292,20,1
4,5,Exp1,low,5,0.958333,120,0.018395,21,1


In [71]:
lm = pg.linear_regression(dat['m_WMCrr'], dat['WMSize_num'])
lm.round(2)

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,5.79,0.24,24.02,0.0,0.57,0.57,5.32,6.27
1,m_WMCrr,-5.06,0.32,-15.93,0.0,0.57,0.57,-5.69,-4.43


In [56]:
pg.rm_anova(data = dat, 
            dv = 'm_WMCrr', 
            within = ['Exp','WMSize'], 
            subject = 'NSub', detailed = True)

/Users/fionazhu/opt/anaconda3/lib/python3.8/site-packages/pingouin/distribution.py:459: UserWarning: Epsilon values might be innaccurate in two-way repeated measures design where each  factor has more than 2 levels. Please  double-check your results.
  warnings.warn("Epsilon values might be innaccurate in "


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,np2,eps
0,Exp,0.330837,3,45,0.110279,15.260634,5.471974e-07,1.672614e-05,0.504306,0.706830
1,WMSize,1.688553,2,30,0.844277,439.695600,5.964274e-23,1.260730e-21,0.967011,0.936301
2,Exp * WMSize,0.022749,6,90,0.003791,2.062507,6.544630e-02,1.446444e-01,0.120879,0.334172


In [57]:
# use the same inputs as rm_anova(), and specify padjust: 'bonf', 'holm', 'fdr_bh' etc.
pg.pairwise_ttests(data = dat.query('Exp == "Exp1"'), 
            dv = 'm_WMCrr',  within = 'WMSize', subject = 'NSub', padjust = 'holm')

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,WMSize,high,low,True,True,-13.603306,15.0,two-sided,7.650379e-10,2.295114e-09,holm,1.263e+07,-3.320000
1,WMSize,high,medium,True,True,-8.177989,15.0,two-sided,6.559945e-07,6.559945e-07,holm,2.586e+04,-1.296554
2,WMSize,low,medium,True,True,12.541506,15.0,two-sided,2.359283e-09,4.718566e-09,holm,4.471e+06,2.481272


## A repeated measures ANOVA of weight of prior with Experiment (1-4) 

In [73]:
#load data
Baypar_WMSize = pd.read_csv('modelrlt/models/log_rstan/rlt/Baypar_WMSize.csv')
Baypar_WMSize.head()

,Unnamed: 0,NSub,Exp,WMSize,wp,InP_Eq,InP_Eq_old,InP_curve,m_predInP_boot,mRP_InP_boot
0,1,1,Exp1,low,0.747986,0.896639,0.915302,0.92,0.896535,0.873041
1,2,2,Exp1,low,0.508092,1.104924,1.138781,1.09,1.093693,1.112962
2,3,3,Exp1,low,0.531333,0.996140,1.015170,1.02,0.974180,0.978668
3,4,4,Exp1,low,0.628105,0.951926,0.973546,0.97,0.942744,0.966582
4,5,5,Exp1,low,0.182516,0.927185,0.965167,0.97,0.901981,0.888706


In [91]:
#significantly larger than 0 in all four experiments (all wps > 0.36, all _ts > 8.14_, all ps < 0.001)
Baypar_WMSize['wp_zero'] = 0
pg.ttest(Baypar_WMSize.query('Exp == "Exp1"').wp, Baypar_WMSize.query('Exp == "Exp1"').wp_zero )

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,16.631705,94,two-sided,9.417278e-30,"[0.52, 0.66]",3.394933,1.949e+26,1.0


In [93]:
pg.ttest(Baypar_WMSize.query('Exp == "Exp2"').wp, Baypar_WMSize.query('Exp == "Exp2"').wp_zero )

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,16.631705,94,two-sided,9.417278e-30,"[0.52, 0.66]",3.394933,1.949e+26,1.0


In [95]:

pg.ttest(Baypar_WMSize.query('Exp == "Exp3"').wp, Baypar_WMSize.query('Exp == "Exp3"').wp_zero )

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,20.764864,94,two-sided,6.896133e-37,"[0.46, 0.55]",4.23861,1.964e+33,1.0


In [94]:

pg.ttest(Baypar_WMSize.query('Exp == "Exp4"').wp, Baypar_WMSize.query('Exp == "Exp4"').wp_zero )

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,14.028673,94,two-sided,8.804067e-25,"[0.35, 0.47]",2.863591,2.685e+21,1.0


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,14.031864,94,two-sided,8.676996e-25,"[0.35, 0.46]",2.864242,2.724e+21,1.0


In [11]:
pg.rm_anova(data = Baypar_WMSize, 
            dv = 'wp', 
            within = ['Exp','WMSize'], 
            subject = 'NSub', detailed = True)

/Users/fionazhu/opt/anaconda3/lib/python3.8/site-packages/pingouin/distribution.py:459: UserWarning: Epsilon values might be innaccurate in two-way repeated measures design where each  factor has more than 2 levels. Please  double-check your results.
  warnings.warn("Epsilon values might be innaccurate in "


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,np2,eps
0,Exp,1.113175,3,45,0.371058,3.145659,3.417078e-02,4.129652e-02,0.173356,0.878036
1,WMSize,0.071793,2,30,0.035896,98.412215,6.629357e-14,4.648021e-08,0.867739,0.506471
2,Exp * WMSize,0.085646,6,90,0.014274,34.047102,3.728908e-21,1.061615e-06,0.694172,0.243137


## A repeated measures ANOVA of Indifference Point 

In [13]:
pg.rm_anova(data = Baypar_WMSize, 
            dv = 'InP_curve', 
            within = ['Exp','WMSize'], 
            subject = 'NSub', detailed = True)

/Users/fionazhu/opt/anaconda3/lib/python3.8/site-packages/pingouin/distribution.py:459: UserWarning: Epsilon values might be innaccurate in two-way repeated measures design where each  factor has more than 2 levels. Please  double-check your results.
  warnings.warn("Epsilon values might be innaccurate in "


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,np2,eps
0,Exp,0.026637,3,45,0.008879,0.304681,0.821848,0.777085,0.019908,0.798034
1,WMSize,0.000929,2,30,0.000465,0.785765,0.464927,0.402201,0.049777,0.561416
2,Exp * WMSize,0.039687,6,90,0.006615,5.309957,0.000100,0.018885,0.261446,0.254211
